In [1]:
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain import OpenAI, VectorDBQA, PromptTemplate

In [100]:
# In this notebook we are loading in a subset of the documents from our developer documentation and
# use embeddings and a vector store to perform a similarity search from a question query to these documents.
# We then use a language model to generate an answer based on the results from the search.

In [77]:
# Parse all documentation files in docs/ directory and create two lists containing the content and metadata

docs = []
metadatas = []

for path, currentDir, files in os.walk('docs/'):
    for file in files:
        file_path = os.path.join(path, file)
        with open(file_path) as f:
            docs.append(f.read())
            metadatas.append({"source": file_path})

In [78]:
print(len(docs))

252


In [79]:
# Read Files

In [80]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

In [81]:
# Create documents

In [82]:
documents = text_splitter.create_documents(docs, metadatas=metadatas)

Created a chunk of size 1337, which is longer than the specified 1000


In [83]:
print(len(documents))

1916


In [85]:
# Create vector store (this step takes around 10 minutes on my Mac M1, but only has to be performed once)
import time

def time_convert(sec):
  mins = sec // 60
  sec = sec % 60
  hours = mins // 60
  mins = mins % 60
  print("Time Lapsed = {0}:{1}:{2}".format(int(hours),int(mins),int(sec)))

# Start stopwatch
start_time = time.time()

embeddings = OpenAIEmbeddings()

# Perform similarity search
docsearch = FAISS.from_documents(documents, embeddings)

# End stopwatch
end_time = time.time()
time_lapsed = end_time - start_time
time_convert(time_lapsed)

Time Lapsed = 0:10:24


In [42]:
# Define QA chain
# Here we connect the calculated vector store and the language model
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch)

In [95]:
# Prompt template (feel free to adapt this one)
template = """
You are an AI assistant for the developer documentation of the eCommerce API Backend Shopware. The documentation is located at https://developer.shopware.com.
You are given the following extracted parts of a long document and a question.
If the question includes a request for code, provide a code block directly from the documentation.
Pay special attention to differ between a Shopware 6 App and a Plugin.
If you don't know the answer, just say "Hmm, I'm not sure." Don't try to make up an answer.
If the question is not about Shopware, politely inform them that you are tuned to only answer questions about Shopware.
Question: {question}
=========
Answer in Markdown:

"""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [101]:
# Evaluate and display result

print(qa.run(prompt.format(question="What is an indexer used for?")))

 An indexer is used to optimize the performance of recurring complex tasks. A good example of this is the cheapest price calculation within Shopware. The indexer will calculate the cheapest price of a product whenever the product is updated by the Data Abstraction Layer (DAL). This means that no new calculation has to be performed when a product is read, and performance during reading is significantly increased. Furthermore, data indexers can make use of the message queue to handle the calculations asynchronously.


In [102]:
# Below are some more examples, feel free to adapt them to test.

print(qa.run(prompt.format(question="What types of filters are there?")))


Filters are used to only find specific data from the database. Shopware 6's data abstraction layer supports multiple types of filters, such as `equals`, `greaterThan`, `lessThan`, `not` and `contains`. You can use these filters together when writing a custom query or when using repositories.


In [103]:
print(qa.run(prompt.format(question="What is the DAL?")))

 The Data Abstraction Layer (DAL) is the data handling layer used in Shopware 6. It is a thin layer that sits between the application and the database. Unlike most other Symfony applications, Shopware 6 uses no ORM but rather a thin Data Abstraction Layer which can be accessed directly.


In [31]:
print(qa.run(prompt.format(question="How do I use the DAL?")))

 To use the Data Abstraction Layer (DAL), you need to understand the concept behind it, which can be found in our documentation [here](../../../../../concepts/framework/data-abstraction-layer). Additionally, you need to read the guide about [reading data](reading-data) and [writing data](writing-data) to understand how to use the DAL for both tasks.


In [32]:
print(qa.run(prompt.format(question="Can I use the DAL to build an API?")))

 Yes, you can use the Data Abstraction Layer (DAL) to build APIs. Please refer to our [API documentation](https://developer.shopware.com/documentation/api) and the [DAL documentation](https://developer.shopware.com/documentation/concepts/framework/data-abstraction-layer) for more information.


In [33]:
print(qa.run(prompt.format(question="Does the DAL support inheritance?")))


Yes, the Data Abstraction Layer supports inheritance for custom entities. You can find more information about this in the [concepts guide about the DAL](../../../../../concepts/framework/data-abstraction-layer#inheritance).


In [36]:
print(qa.run(prompt.format(question="How about translations?")))


You can add translations to entities. For more information, check out the guide on [Adding data translations](../../plugins-backend/add-data-translations). There is also a video from our free online training ["Backend Development"](https://academy.shopware.com/courses/shopware-6-backend-development-with-jisse-reitsma) about [Translating your entity](https://www.youtube.com/watch?v=FfqxfQl3I4w).


In [37]:
print(qa.run(prompt.format(question="How can I add attributes to existing entites")))


You can add attributes to existing entities by creating a new database table with a plugin migration. Please refer to the [Plugin migrations](../../plugin-fundamentals/database-migrations) guide for more information.


In [38]:
print(qa.run(prompt.format(question="What is the best way to extend an entity?")))


The best way to extend an entity is to create a custom extension class that extends from the abstract `Shopware\Core\Framework\DataAbstractionLayer\EntityExtension` class and add your fields in the `extendFields` method. You can see an example of this below. 

<CodeBlock title="<plugin root>/src/Extension/Content/Product/CustomExtension.php">

```php
<?php declare(strict_types=1);

namespace Swag\BasicExample\Extension\Content\Product;

use Shopware\Core\Content\Product\ProductDefinition;
use Shopware\Core\Framework\DataAbstractionLayer\EntityExtension;
use Shopware\Core\Framework\DataAbstractionLayer\FieldCollection;

class CustomExtension extends EntityExtension
{
    public function extendFields(FieldCollection $collection): void
    {
        $collection->add(
            // new fields here
        );
    }

    public function getDefinitionClass(): string
    {
        return ProductDefinition::class;
    }
}
```



In [87]:
print(qa.run(prompt.format(question="""
I need a filter that searches for invoices. This option is not available in search preferences. Is it possible to search by invoice number?
""")))

 Yes, it is possible to search by invoice number. You can apply a `EqualsFilter` to the `Criteria` object, which accepts a field name and the value to search for. You can find the `EqualsFilter` here `Shopware\Core\Framework\DataAbstractionLayer\Search\Filter\EqualsFilter`.

Example:

```php
public function readData(Context $context): void
{
    $criteria = new Criteria();
    $criteria->addFilter(new EqualsFilter('invoice_number', '12345'));

    $invoices = $this->invoiceRepository->search($criteria, $context);
}
```

This example will search for all invoices with the invoice number `12345` and return an `EntitySearchResult` containing all matched invoices. Since the `EntitySearchResult` is extending the `EntityCollection`, which is iterable, you could just iterate over the results using a `foreach`.


In [98]:
print(qa.run(prompt.format(question="""
I need to retrieve a customer connected to the storefront backend side to reward him in different ways.

I created a plugin that extends the plugin.class of the plugins system.

It fetches the customer on the store api using the route store-api/account/customer then it sends to my backend its identifier. I also resolve the shop_url of the admin api with window.location.protocol and window.location.hostname...

This seems to me not secured or accurate (the domain can be different from the sales channel to the admin api) and I would like to know if it would be possible to fetch a secured unique customer's token that would allow me to resolve both the shop_url and the customer's identifier.

I cannot find anything in the documentation that would help me securing that part of my app.

Thanks.
""")))



You can use the `Shopware\Core\System\SalesChannel\SalesChannelContext` service to retrieve a customer's token and identifier. You can inject this service into your plugin class, and use it to get the customer's token:

```php
<?php

use Shopware\Core\System\SalesChannel\SalesChannelContext;

// ...

public function __construct(SalesChannelContext $salesChannelContext)
{
    $this->salesChannelContext = $salesChannelContext;
}

public function getCustomerToken()
{
    $token = $this->salesChannelContext->getCustomer()->getToken();

    return $token;
}
```

You can use this token to securely access the shop's url, as well as the customer's identifier.
